<a href="https://colab.research.google.com/github/Tomawock/NLP_Attack/blob/main/Augmentation_test_ATE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install textattack

In [2]:
import random
import timeit

In [ ]:
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')

dataset = pd.read_json('/content/drive/Shareddrives/Deep Learning/datasets/ATE_ABSITA/ATE_ABSITA_training_set/ate_absita_training.ndjson'
                       , lines=True)

In [ ]:
dataset.drop(columns=['id_sentence'], inplace=True)

In [17]:
print(f'Contains {len(dataset)} sentences')
dataset.head()

Contains 3054 sentences


,sentence,score,polarities,aspects_position,aspects
0,"Ottimo prodotto di marca, la qualità é veramen...",5,"[[0, 0], [0, 1], [1, 0]]","[[120, 142], [71, 79], [29, 36]]","[provvisto di una tasca, capiente, qualità]"
1,Ottimo rasoio dal semplice utilizzo. Rade molt...,5,"[[1, 0], [1, 0], [1, 0], [1, 0]]","[[18, 26], [37, 41], [79, 86], [99, 105]]","[semplice, Rade, Pratico, pulire]"
2,Un quarto delle dimensioni dello Show original...,5,"[[1, 0], [1, 0], [1, 0], [0, 0]]","[[118, 132], [51, 62], [65, 70], [16, 26]]","[modalità notte, prestazioni, suono, dimensioni]"
3,Il prodotto si presenta esattamente come in fo...,5,"[[1, 0], [1, 0], [1, 0]]","[[147, 158], [132, 140], [24, 48]]","[vestibilità, capienza, esattamente come in foto]"
4,"Superlativa, velocità in scrittura superiore a...",5,"[[1, 0]]","[[13, 21]]",[velocità]


###Sono presenti solo 20 duplicati. Si decide di non rimuoverli

In [14]:
len(dataset.groupby('sentence'))

3034

In [40]:
aug = dataset.copy()

##Augmentation Phase

###WordNet

In [60]:
from textattack.transformations import WordSwapWordNet

from textattack.constraints.pre_transformation import RepeatModification
from textattack.constraints.pre_transformation import StopwordModification

from textattack.augmentation import Augmenter
random.seed(33)


transformation = WordSwapWordNet(language='ita')
constraints = [RepeatModification(), StopwordModification()]

my_items = []
num_tras_to_add = 2

start_time = timeit.default_timer()
augmenter = Augmenter(transformation=transformation,
                      constraints=constraints,
                      pct_words_to_swap=1,
                      transformations_per_example=num_tras_to_add)

for row in aug.head(10).itertuples():
  result=augmenter.augment(row.sentence)
  my_items.append({'sentence': row.sentence,
                   'score': row.score,
                   'polarities': row.polarities,
                   'aspects_position': row.aspects_position,
                   'aspects': row.aspects})
  for i in range(num_tras_to_add):
    ### QUI MODIFICHE
    my_items.append({'sentence': result[i],
                     'score': row.score,
                     'polarities': row.polarities,
                     'aspects_position': row.aspects_position,
                     'aspects': row.aspects})

elapsed = timeit.default_timer() - start_time
print(f'TIME OF EXECUTION={elapsed}')
augmeted_new = pd.DataFrame(data=my_items, columns=aug.columns)
print(f"COLONNE:{augmeted_new.columns}")
print(f"DIMENSIONE:{augmeted_new.shape}")

TIME OF EXECUTION=0.5047117140002229
COLONNE:Index(['sentence', 'score', 'polarities', 'aspects_position', 'aspects'], dtype='object')
DIMENSIONE:(6, 5)


In [ ]:
augmeted_new

###WordEmbedding

In [ ]:
from textattack.shared import WordEmbedding

class WordEmbeddingIta(WordEmbedding):
    def counterfitted_GLOVE_embedding():
      """Returns a prebuilt counter-fitted GLOVE word embedding proposed by
      "Counter-fitting Word Vectors to Linguistic Constraints" (Mrkšić et
      al., 2016)"""
      if (
          "textattack_counterfitted_GLOVE_embedding" in utils.GLOBAL_OBJECTS
          and isinstance(
              utils.GLOBAL_OBJECTS["textattack_counterfitted_GLOVE_embedding"],
              WordEmbedding,
          )
      ):
          # avoid recreating same embedding (same memory) and instead share across different components
          return utils.GLOBAL_OBJECTS["textattack_counterfitted_GLOVE_embedding"]

      word_embeddings_folder = "paragramcf"   # NOSTRO PATH
      word_embeddings_file = "paragram.npy"
      word_list_file = "wordlist.pickle"
      mse_dist_file = "mse_dist.p"
      cos_sim_file = "cos_sim.p"
      nn_matrix_file = "nn.npy"

      # Download embeddings if they're not cached.
      word_embeddings_folder = os.path.join(WordEmbedding.PATH,
                                            word_embeddings_folder)

      word_embeddings_folder = utils.download_if_needed(word_embeddings_folder)
      # Concatenate folder names to create full path to files.
      word_embeddings_file = os.path.join(word_embeddings_folder,
                                          word_embeddings_file)

      word_list_file = os.path.join(word_embeddings_folder, word_list_file)
      mse_dist_file = os.path.join(word_embeddings_folder, mse_dist_file)
      cos_sim_file = os.path.join(word_embeddings_folder, cos_sim_file)
      nn_matrix_file = os.path.join(word_embeddings_folder, nn_matrix_file)

      # loading the files
      embedding_matrix = np.load(word_embeddings_file)
      word2index = np.load(word_list_file, allow_pickle=True)
      index2word = {}
      for word, index in word2index.items():
          index2word[index] = word
      nn_matrix = np.load(nn_matrix_file)

      embedding = WordEmbedding(embedding_matrix, word2index, index2word, nn_matrix)

      with open(mse_dist_file, "rb") as f:
          mse_dist_mat = pickle.load(f)
      with open(cos_sim_file, "rb") as f:
          cos_sim_mat = pickle.load(f)

      embedding._mse_dist_mat = mse_dist_mat
      embedding._cos_sim_mat = cos_sim_mat

      utils.GLOBAL_OBJECTS["textattack_counterfitted_GLOVE_embedding"] = embedding

      return embedding

In [ ]:
from textattack.augmentation import EmbeddingAugmenter

num_tras_to_add = 2
augmenter = EmbeddingAugmenter(pct_words_to_swap=1,
                               transformations_per_example=num_tras_to_add,
                               embedding=WordEmbedding.counterfitted_GLOVE_embedding())

In [ ]:
augmeted_new